# RealClearPolitics: 2022 Senate polls

#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_grid as altgrid
import numpy as np
import us
import urllib.request, json
import glob
import os
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import os
import time

In [3]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.set_option("display.max_colwidth", None)

In [5]:
today = pd.to_datetime("today").strftime("%Y-%m-%d")

In [6]:
today

'2022-10-26'

---

## Harvest data 

#### First, get all the poll page urls from the 2022 landing page

In [7]:
url = "https://www.realclearpolitics.com/epolls/latest_polls/senate/"

In [8]:
path = "/Users/stiles/github/chromedriver"
s = Service(path)
driver = webdriver.Chrome(service=s)

In [9]:
url = "https://www.realclearpolitics.com/epolls/latest_polls/senate/"
driver.get(url)
html = driver.page_source

In [10]:
soup = BeautifulSoup(html, "html.parser")
links = soup.findAll("a")

In [11]:
data = soup.findAll("td", attrs={"class": "lp-race"})

In [12]:
links = []

for td in data:
    page_links = td.findAll("a")
    for a in page_links:
        links.append("https://www.realclearpolitics.com" + a["href"])

In [13]:
links = list(set(links))

In [14]:
path = "/Users/stiles/github/chromedriver"
s = Service(path)
page_driver = webdriver.Chrome(service=s)

In [15]:
dfs = []

for page_link in links:
    page_driver.get(page_link)
    page_html = page_driver.page_source
    page_soup = BeautifulSoup(page_html, "html.parser")
    dfs.append((pd.read_html(page_driver.page_source)[0]).assign(race=page_link))

In [16]:
df = pd.concat(dfs)

In [17]:
df["race"] = (
    df["race"]
    .str.replace(
        "https://www.realclearpolitics.com/epolls/2022/senate/", "", regex=False
    )
    .str.replace(".html", "", regex=False)
)
df.drop(["Sample", "MoE"], axis=1, inplace=True)

In [18]:
src = df[df["Poll"].str.contains("RCP")]

In [19]:
src

,Poll,Date,Kennedy (R) *,Mixon (D),Chambers (D),Steib (D),Spread,race,Hassan (D) *,Bolduc (R),Kelly (D) *,Masters (R),Victor (L),Blumenthal (D) *,Levy (R),Thune (R) *,Bengs (D),Johnson (R) *,Barnes (D),Murray (D) *,Smiley (R),Mullin (R),Horn (D),Schumer (D) *,Pinion (R),Laxalt (R),Cortez Masto (D) *,Wyden (D) *,Perkins (R),Budd (R),Beasley (D),Padilla (D) *,Meuser (R),Duckworth (D) *,Salvi (R),Lee (R) *,McMullin (I),Schmitt (R),Valentine (D),Warnock (D) *,Walker (R),Bennet (D) *,O'Dea (R),Lankford (R) *,Rubio (R) *,Demings (D),Fetterman (D),Oz (R),Grassley (R) *,Franken (D),Vance (R),Ryan (D)
0,RCP Average,10/2 - 10/23,NaN,NaN,NaN,NaN,Hassan +3.6,nh/new-hampshire-senate-bolduc-vs-hassan-7379,49.3,45.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,RCP Average,10/10 - 10/23,NaN,NaN,NaN,NaN,Blumenthal +11.0,ct/connecticut_senate_levy_vs_blumenthal-7686,NaN,NaN,NaN,NaN,NaN,52.7,41.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,RCP Average,10/3 - 10/17,NaN,NaN,NaN,NaN,Johnson +2.7,wi/wisconsin_senate_johnson_vs_barnes-7758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.7,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,RCP Average,9/30 - 10/19,NaN,NaN,NaN,NaN,Murray +8.5,wa/washington_senate_smiley_vs_murray-7400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,41.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,RCP Average,10/3 - 10/18,NaN,NaN,NaN,NaN,Schumer +13.0,ny/new_york_senate_schumer_vs_pinion-7771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.7,39.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,RCP Average,10/14 - 10/20,NaN,NaN,NaN,NaN,Laxalt +0.3,nv/nevada_senate_laxalt_vs_cortez_masto-7392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.3,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,RCP Average,10/10 - 10/22,NaN,NaN,NaN,NaN,Budd +4.5,nc/north_carolina_senate_budd_vs_beasley-7588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.5,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,RCP Average,8/8 - 9/27,NaN,NaN,NaN,NaN,Schmitt +11.0,mo/missouri_senate_schmitt_vs_busch_valentine-7682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,RCP Average,10/13 - 10/23,NaN,NaN,NaN,NaN,Warnock +0.5,ga/georgia-senate-walker-vs-warnock-7329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,46.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,RCP Average,10/3 - 10/6,NaN,NaN,NaN,NaN,Bennet +7.5,co/colorado_senate_odea_vs_bennet-7773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.5,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
src_melted = src.melt(
    value_vars=[
        "Bennet (D) *",
        "O'Dea (R)",
        "Lee (R) *",
        "McMullin (I)",
        "Wyden (D) *",
        "Perkins (R)",
        "Laxalt (R)",
        "Cortez Masto (D) *",
        "Padilla (D) *",
        "Meuser (R)",
        "Kelly (D) *",
        "Masters (R)",
        "Victor (L)",
        "Warnock (D) *",
        "Walker (R)",
        "Schumer (D) *",
        "Pinion (R)",
        # "Van Hollen (D) *",
        # "Chaffee (R)",
        "Hassan (D) *",
        "Bolduc (R)",
        "Vance (R)",
        "Ryan (D)",
        "Schmitt (R)",
        "Valentine (D)",
        "Johnson (R) *",
        "Barnes (D)",
        "Budd (R)",
        "Beasley (D)",
        # "Moran (R) *",
        # "Holland (D)",
        "Thune (R) *",
        "Bengs (D)",
        "Rubio (R) *",
        "Demings (D)",
        "Grassley (R) *",
        "Franken (D)",
        "Duckworth (D) *",
        "Salvi (R)",
        # "Boozman (R) *",
        # "James (D)",
        "Mullin (R)",
        "Horn (D)",
        "Lankford (R) *",
        "Kennedy (R) *",
        "Mixon (D)",
        "Chambers (D)",
        "Steib (D)",
        "Fetterman (D)",
        "Oz (R)",
        "Murray (D) *",
        "Smiley (R)",
        "Blumenthal (D) *",
        "Levy (R)",
    ],
    id_vars=["Date", "Poll", "Spread", "race"],
).dropna(subset="value")

In [22]:
src_melted["state"] = src_melted["race"].str.split("/", expand=True)[0].str.upper()

In [23]:
src_melted[["candidate", "party"]] = src_melted["variable"].str.split(
    " \(", expand=True
)

In [24]:
src_melted[["party", "incumbent"]] = src_melted["party"].str.split(")", expand=True)

In [25]:
src_melted.drop(["race", "variable", "Spread", "Poll"], axis=1, inplace=True)

In [26]:
src_melted.head()

,Date,value,state,candidate,party,incumbent
9,10/3 - 10/6,49.5,CO,Bennet,D,*
23,10/3 - 10/6,42.0,CO,O'Dea,R,
89,10/14 - 10/20,46.3,NV,Laxalt,R,
103,10/14 - 10/20,46.0,NV,Cortez Masto,D,*
153,10/11 - 10/18,45.8,AZ,Kelly,D,*


In [27]:
src_wide = src_melted.pivot_table(
    index=["state", "Date"], values="value", columns="party"
).reset_index()

In [28]:
src_wide.columns = src_wide.columns.str.lower()

In [29]:
src_wide["year"] = "2022"

In [30]:
postal_to_name = us.states.mapping("abbr", "name")
src_wide["state"] = src_wide["state"].map(postal_to_name)

In [31]:
df = src_wide.drop(["date"], axis=1).copy()

In [32]:
df.rename(columns={"d": "dem_polling", "r": "gop_polling"}, inplace=True)

In [33]:
df["dem_polling_margin"] = (df["dem_polling"] - df["gop_polling"]).round(2)
df["gop_polling_margin"] = (df["gop_polling"] - df["dem_polling"]).round(2)

In [34]:
df["description"] = "RCP polling average"

In [36]:
df["date"] = "2022-10-26"

---

#### How have things changed? 

In [37]:
df.head()

party,state,dem_polling,gop_polling,year,dem_polling_margin,gop_polling_margin,description,date
0,Arizona,45.8,43.3,2022,2.5,-2.5,RCP polling average,2022-10-26
1,Colorado,49.5,42.0,2022,7.5,-7.5,RCP polling average,2022-10-26
2,Connecticut,52.7,41.7,2022,11.0,-11.0,RCP polling average,2022-10-26
3,Florida,41.8,49.3,2022,-7.5,7.5,RCP polling average,2022-10-26
4,Georgia,47.0,46.5,2022,0.5,-0.5,RCP polling average,2022-10-26


In [38]:
old_df = pd.read_csv("data/processed/2022_polling_average_states_RCP_oct_5.csv")

In [39]:
old_df["date"] = "2022-10-05"

In [40]:
thennow_df = pd.concat([old_df, df]).reset_index(drop=True)

In [41]:
thennow_df["display_date"] = pd.to_datetime(thennow_df["date"]).dt.strftime("%b. %-d")
# thennow_df["date"] = pd.to_datetime(thennow_df["date"])

In [42]:
thennow_df.head()

,state,dem_polling,gop_polling,year,dem_polling_margin,gop_polling_margin,description,date,display_date
0,Arizona,48.7,44.8,2022,3.9,-3.9,RCP polling average,2022-10-05,Oct. 5
1,Colorado,47.0,38.0,2022,9.0,-9.0,RCP polling average,2022-10-05,Oct. 5
2,Connecticut,53.0,38.7,2022,14.3,-14.3,RCP polling average,2022-10-05,Oct. 5
3,Florida,43.0,47.0,2022,-4.0,4.0,RCP polling average,2022-10-05,Oct. 5
4,Georgia,48.0,44.2,2022,3.8,-3.8,RCP polling average,2022-10-05,Oct. 5


In [45]:
chart = (
    alt.Chart()
    .mark_line(color="#1851ac")
    .encode(
        x=alt.X("display_date:O", title="", sort="-x"),
        y=alt.Y("dem_polling_margin", title="", axis=alt.Axis()),
    )
)

text = (
    alt.Chart()
    .mark_text(dx=0, dy=-8, color="black")
    .encode(
        x=alt.X("display_date", sort="-x"),
        y=alt.Y("dem_polling_margin"),
        text=alt.Text("dem_polling_margin"),
    )
)

alt.layer(chart, text, data=thennow_df,).properties(width=100, height=120,).facet(
    facet=alt.Facet("state", title=" "), columns=7
).properties(
    title="Percentage point change in Democrats' polling average margin since Oct. 5"
)

alt.FacetChart(...)

## Exports

In [ ]:
df.to_csv(f"data/processed/2022_polling_average_states_RCP_{today}.csv", index=False)
df.to_csv("data/processed/2022_polling_average_states_RCP.csv", index=False)